In [15]:
from __future__ import print_function
import sys
import cv2
import tracker 
from utils import select_bounding_box
from video_read_thread import FileVideoStream
import time
import imutils
from random import randint

In [20]:
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml'
face_detection = cv2.CascadeClassifier(detection_model_path)

def get_faces_from_haar(frame):
    bboxes=[]
    colors=[]
    frame=imutils.resize(frame,width=500)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
    faces=face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    print(len(faces)," faces found in the frame")
    if len(faces)>0:
        for face in faces:
            (x,y,w,h) = faces[0]
            bboxes.append((x,y,w,h))
    return bboxes,colors

In [21]:
# Set video to load
videoPath = r"C:/Users/HP/Downloads/VID20200314183342.mp4"
# Create a video capture object to read videos
cap=FileVideoStream(videoPath).start() 
# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
time.sleep(1.0)
#if not success:
#    print('Failed to read video')
#    sys.exit(1)

In [22]:
#bboxes,colors=select_bounding_box(frame)
bboxes,colors=get_faces_from_haar(frame)
multiTracker = tracker.create_multiTracker(bboxes,frame)

0  faces found in the frame
Tracker Created ...


In [19]:
# Process video and track objects
while cap.more():
    success, frame = cap.read()
    if not success:
        break
# get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)
# draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
# show frame
    cv2.imshow('MultiTracker', frame)
# quit on ESC button
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Esc pressed
        break
cap.stop()
cv2.destroyAllWindows()